### **Cleansing Data**

Install libraries and import libraries

In [ ]:
%pip install pandas numpy geopy

In [ ]:
import pandas as pd
import re
import html
from geopy.geocoders import Nominatim

Read CSV File

In [ ]:
df = pd.read_csv('crossref_papers.csv', on_bad_lines='skip')

Fix HTML tag in title and abstract

In [ ]:
# Function to remove HTML tags and decode HTML entities
def clean_html(text):
    if isinstance(text, str):  # Check if the text is a string
        # Decode HTML entities
        text = html.unescape(text)
        # Remove all HTML tags
        return re.sub(r'<[^>]*>', '', text).strip()  # Remove anything between <>
    return text  # If it's not a string, return as is

# Apply cleaning to the 'title' and 'abstract' columns
df['title'] = df['title'].apply(clean_html)
df['abstract'] = df['abstract'].apply(clean_html)

Fix title and abstract string error

In [ ]:
# Function to remove paragraphs and line breaks
def clean_paragraphs(text):
    # Replace line breaks with a single space
    text = text.replace('\n', ' ').replace('\r', '')
    # Remove extra spaces between words
    text = ' '.join(text.split())
    return text

# Apply the clean_abstract function to the 'abstract' column
df['title'] = df['title'].apply(clean_paragraphs)
df['abstract'] = df['abstract'].apply(clean_paragraphs)

Cut off the year that under 2019

In [ ]:
# Filter rows where the year is less than or equal to 2019
df_deleted = df[df['year'] <= 2019]

# Filter rows where the year is greater than 2019
df_remaining = df[df['year'] > 2019]

# Display the count of deleted and remaining rows
print(f"Number of Rows Deleted: {len(df_deleted)}")
print(f"Number of Rows Remaining: {len(df_remaining)}")

Edit Abstract Format that start with text Abstract

In [ ]:
# Function to remove 'Abstract' or 'abstract' from the start of the text
def remove_abstract(text):
    if isinstance(text, str):  # Check if the text is a string
        # Check if the text starts with "Abstract" or "abstract"
        if text.lower().startswith('abstract'):
            # Remove the word 'Abstract' (or 'abstract') from the start and any following space
            text = text[8:].lstrip()  # Start from the 9th character to remove 'Abstract' + space
    return text

# Apply the remove_abstract function to the 'abstract' column
df_remaining['abstract'] = df_remaining['abstract'].apply(remove_abstract)

Drop the duplicate rows

In [ ]:
df_remaining = df_remaining.drop_duplicates()

Check affiliation error

In [ ]:
df_remaining = df_remaining[~df_remaining['affiliation'].str.lower().isin(['independent researcher', 'independent consultant', 'independent'])]

Save the Cleaned Data to CSV

In [ ]:
# Save the remaining rows to a new CSV file
df_remaining.to_csv('cleaned_file.csv', index=False)
print("Cleaning done! Final file saved as 'cleaned_file.csv'")

Make only affiliations for find latitude, longitude

In [ ]:
# Read the cleaned data file
df = pd.read_csv('cleaned_file.csv')

# Extract unique affiliations
df = df['affiliation'].drop_duplicates()
df = pd.DataFrame(df)      

Format affiliation to latitude, longitude

In [ ]:
# Geolocator setup
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to get latitude and longitude
def get_coordinates(x):
    try:
        location = geolocator.geocode(x)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching coordinates for {x}: {e}")
        return None, None

# Apply the function to the affiliation column
df['latitude'], df['longitude'] = zip(*df['affiliation'].apply(get_coordinates))

# Drop rows with NaN values in latitude or longitude
df_cleaned = df.dropna(subset=['latitude', 'longitude'])

# Save the cleaned DataFrame to a CSV file
df_cleaned.to_csv('affiliations_with_coordinates.csv', index=False)

Merge affiliation_with_coordinates to the cleaned_file to get final result for Training

In [ ]:
# Read the CSV files
cleaned_file = pd.read_csv('cleaned_file.csv')
merged_file = pd.read_csv('affiliations_with_coordinates.csv')

# Merge the two DataFrames on the 'affiliation' column
final_df = pd.merge(cleaned_file, merged_file, on='affiliation', how='left')

# Drop rows where latitude or longitude is missing
final_df = final_df.dropna(subset=['latitude', 'longitude'])

# Save the merged and cleaned DataFrame to a new CSV file
final_df.to_csv('Final.csv', index=False)

df = pd.read_csv('Final.csv')
df.shape